In [1]:
import torch
import torch.nn as nn
import tiktoken


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))
    
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            # Hidden layer neurons = 4 * len(emb dim)
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            # Non linear activation function
            GELU(),
            # Must squish output back down to emb dim so that each transformer block will output same length tensor
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [4]:
# Attention class
class MultiHeadAttention(nn.Module):
    #Constructor Inputs:d in is the embedding vector length, d out is the context vector length, context length is 
    # the number of tokens, dropout is the rate of data points left out, num_heads is the number of attention heads you want running
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
# forward is called whenever class is called on a certain bacthed data (x is the batched data)
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [5]:
class GPTModel(nn.Module):
    def __init__(self, cfg):    # cfg refers to the gpt config dictionary above
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]) # makes/unpacks a list of transformer blocks, stuck together as modules
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        # Makes embedding vectors from input ids
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        # Transformer stuff
        x = self.drop_emb(x)
        # all the transformer blocks
        x = self.trf_blocks(x)
        #Final normalization step
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits   # tensor of scores like probabilities: logits = log of the odds

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.att = MultiHeadAttention(
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"]
            )
        self.dropout = nn.Dropout(cfg["drop_rate"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.feedforward = FeedForward(cfg)

    def forward(self, x):
        #Attention chunk of the transformer:

        #store original x as variable to add back later in shortcut step
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        s = self.dropout(x)
        x = x + shortcut

        #Feed forward chunk of the transformer:

        shortcut = x
        x = self.norm2(x)
        x = self.feedforward(x)
        x = self.dropout(x)
        s = x + shortcut

        return x

# Keeps the numbers from getting wild
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim)) # make scale and shift trainable
        self.shift = nn.Parameter(torch.zeros(emb_dim)) # Start as 1 and 0, so they may learn to undo the normalization?

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        #Computing z scores
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # eps helps avoid dividing by super small numbers (unstable)
        #self.scale and self.shift are learnable: model can learn to undo normalization
        return self.scale * norm_x + self.shift 
        

In [8]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

In [9]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.3015, -0.4455, -0.7185,  ...,  0.3172, -0.0602, -0.2457],
         [ 0.1608, -0.2113, -0.5270,  ...,  0.9612,  0.0048,  0.3928],
         [ 0.2110, -0.3303, -0.7866,  ...,  0.5302,  0.0784, -0.1427],
         [ 0.2396, -0.1760,  0.1579,  ...,  0.6029,  0.1160,  0.0709]],

        [[ 0.0633,  0.0273,  0.6129,  ..., -0.0104, -0.8202,  0.7848],
         [-0.0677, -0.2861,  0.4478,  ..., -0.0016, -0.8843,  0.5654],
         [-0.1341,  0.4114,  0.2199,  ..., -0.1777, -0.3294,  0.9648],
         [-0.2262,  0.3596,  0.3515,  ...,  0.2644, -1.1025,  1.2120]]],
       grad_fn=<UnsafeViewBackward0>)


## Number of Parameters in Model

In [10]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [11]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [12]:
total_params_gpt2 = (
    total_params - sum(p.numel()
    for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
      f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 124,412,160


## Number of Parameters in Feed Forward

In [14]:
test = FeedForward(GPT_CONFIG_124M)

In [20]:
total_params = sum(p.numel() for p in test.parameters())
print(f"Total number of parameters in FeedFoward: {total_params:,}")


Total number of parameters in FeedFoward: 4,722,432


## Number of Parameters in Multi-Head Attention

In [27]:
test2 = MultiHeadAttention(d_in = GPT_CONFIG_124M["emb_dim"], 
            d_out = GPT_CONFIG_124M["emb_dim"], 
            context_length = GPT_CONFIG_124M["context_length"], 
            dropout = GPT_CONFIG_124M["drop_rate"], 
            num_heads = GPT_CONFIG_124M["n_heads"])

In [29]:
total_params = sum(p.numel() for p in test2.parameters())
print(f"Total number of parameters in MultiHeadAttention: {total_params:,}")


Total number of parameters in MultiHeadAttention: 2,360,064


### There is slightly over twice as many parameters in feedforward than in MultiHeadAttention

# Exercise 4.2

In [39]:
GPT_CONFIG_354M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 1024,         # Embedding dimension
    "n_heads": 16,          # Number of attention heads
    "n_layers": 24,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}


In [40]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

In [41]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_354M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.2119, -0.1909,  0.7996,  ...,  0.5387, -0.6185, -0.6179],
         [-0.1372, -0.5443,  0.5909,  ...,  1.1473, -0.2671, -0.3699],
         [ 0.3345, -0.5944,  0.4180,  ...,  0.8447, -0.2577, -0.2244],
         [-0.0647, -0.0922,  0.5748,  ...,  0.8746, -0.7511, -0.8036]],

        [[ 0.5626, -0.5387,  0.5413,  ...,  0.0332, -0.2636, -0.4786],
         [ 0.4746, -0.1023,  0.5805,  ..., -0.1657, -0.2552, -0.7436],
         [ 0.8460, -0.5311,  1.1762,  ...,  0.2646,  0.2058, -0.3987],
         [ 0.3080, -0.6959,  1.3221,  ...,  0.2868, -0.0460, -0.1332]]],
       grad_fn=<UnsafeViewBackward0>)


In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_354M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.2119, -0.1909,  0.7996,  ...,  0.5387, -0.6185, -0.6179],
         [-0.1372, -0.5443,  0.5909,  ...,  1.1473, -0.2671, -0.3699],
         [ 0.3345, -0.5944,  0.4180,  ...,  0.8447, -0.2577, -0.2244],
         [-0.0647, -0.0922,  0.5748,  ...,  0.8746, -0.7511, -0.8036]],

        [[ 0.5626, -0.5387,  0.5413,  ...,  0.0332, -0.2636, -0.4786],
         [ 0.4746, -0.1023,  0.5805,  ..., -0.1657, -0.2552, -0.7436],
         [ 0.8460, -0.5311,  1.1762,  ...,  0.2646,  0.2058, -0.3987],
         [ 0.3080, -0.6959,  1.3221,  ...,  0.2868, -0.0460, -0.1332]]],
       grad_fn=<UnsafeViewBackward0>)


In [43]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 406,212,608


In [44]:
total_params_gpt2 = (
    total_params - sum(p.numel()
    for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
      f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 354,749,440


### There are 354,749,440 parameters in GPT-2 Medium

In [50]:
GPT_CONFIG_773M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 1280,         # Embedding dimension
    "n_heads": 20,          # Number of attention heads
    "n_layers": 36,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [51]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_773M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape
)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])


In [52]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 838,220,800


In [53]:
total_params_gpt2 = (
    total_params - sum(p.numel()
    for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
      f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 773,891,840


### There are 773,891,840 in GPT-2 Large

In [64]:
GPT_CONFIG_1B = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 1600,         # Embedding dimension
    "n_heads": 25,          # Number of attention heads
    "n_layers": 48,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [65]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_1B)

In [66]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 1,637,792,000


In [67]:
total_params_gpt2 = (
    total_params - sum(p.numel()
    for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
      f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 1,557,380,800


### There are 1,557,380,800 billions parameters in GPT-2 XL